In [1]:
from pathlib import Path

# Make an ExampleScanConfig instance and use it in a GridScanGeneration Task

In [2]:
import obi_one as obi

initialize = obi.ExampleScanConfig.Initialize(morphology_path=obi.NamedPath(name='test', path='test'), param_a=1)
example_scan_config = obi.ExampleScanConfig(initialize=initialize)
grid_scan_generation = obi.GridScanGeneration(scan_config=example_scan_config, output_root=Path("output"))
single_configs, _ = grid_scan_generation.execute()

# Execute the ExampleTask for each of the generated single configs

In [3]:
for single_config in single_configs:
    example_task = obi.ExampleTask(config=single_config)
    example_task.execute()

Scan wrapper

In [4]:
scan_wrapper = obi.ScanWrapper(scan_config=example_scan_config, 
            scan_generation_type=obi.GridScanGeneration)

# scan_wrapper.generate_scan()


